In [1]:
import time
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from utils import data, plot, evaluate
import models
from models import GRU, LSTM

# Set device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Load Data

In [4]:
data_dir = 'data/CMAPSS'
subset = 1
train, test, header, n_meta, n_sensors, n_settings = data.load_cmapss_subset(
    data_dir, subset)

params = {
    'drop_sensors': [1, 5, 6, 10, 16, 18, 19],
    'include_settings': None,
    'smooth': 0,
    'lag': 20,
    'clip': 125,
    'cv_folds': None,
    'return_cols': True,
    'reshape_2d': True
}
X, y, _, cols = data.build_dataset(train, **params)
X_test, y_test, _, cols = data.build_dataset(test, **params, test=True)

y = np.expand_dims(y, axis=1)
y_test = np.expand_dims(y_test, axis=1)
print(X.shape, y.shape)

batch_size = 2048

train_data = TensorDataset(
    torch.from_numpy(X),
    torch.from_numpy(y))
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)

test_data = TensorDataset(
    torch.from_numpy(X_test),
    torch.from_numpy(y_test))
test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size, drop_last=False)

(18631, 14, 21) (18631, 1)


In [5]:
print('GRU Model\n------------------')

model_params = {
    'batch_size': batch_size,
    'learning_rate': 0.001,
    'epochs': 1,
    'model_type': 'GRU'
}

gru_model = models.train(device, train_loader, **model_params)
print('---')
gru_outputs = models.predict(device, gru_model, X_test, y_test)
metrics = evaluate.metrics(y_test, gru_outputs)
print('Test Set - MAPE={:.3f} - MAE={:.3f} - MSE={:.3f} - RMSE={:.3f} - R2={:.3f}'.format(*metrics))

GRU Model
------------------
Starting Training of GRU model
Epoch 1/1 Done, Total Loss: 8186.889539930556
Time Elapsed for Epoch: 11.455600494984537 seconds
Total Training Time: 11.455600494984537 seconds
---
Test Set - MAPE=0.889 - MAE=71.301 - MSE=6810.675 - RMSE=82.527 - R2=-2.944


In [ ]:
print('LSTM Model\n------------------')

model_params = {
    'batch_size': batch_size,
    'learning_rate': 0.001,
    'epochs': 1,
    'model_type': 'LSTM'
}

lstm_model = models.train(device, train_loader, **model_params)
print('---')
lstm_outputs = models.predict(device, gru_model, X_test, y_test)
metrics = evaluate.metrics(y_test, lstm_outputs)
print('Test Set - MAPE={:.3f} - MAE={:.3f} - MSE={:.3f} - RMSE={:.3f} - R2={:.3f}'.format(*metrics))

LSTM Model
------------------
Starting Training of LSTM model
